In [11]:
import os
import numpy as np
import pandas as pd

In [12]:
import sys

print(sys.executable)

d:\edydu\Anaconda\python.exe


### Standardization of SUPERCIAS's data.
The objective of this notebook is to automate the preprocessing and generation of the financial statements of all the companies listed in the Superintendencia de Compañías, Valores y Seguros. The information is quite complete, but the aggregated accounts tend to be imperfectly registered, which is why it's necessary to fill them automatically using the values of the accounts that compose them.

This task can be done using the IFRS manual developed by SUPERCIAS; which details with accuracy which accounts compose the faulty aggregated variables before mentioned. The general idea is to sum up those smaller accounts to get a better version of the faulty variables.

### Balance Sheet
Superintendencia de Compañías, Valores y Seguros (SUPERCIAS) demands three financial statements from ecuadorian companies: Balance Sheet, Income Statement and Cash Flow Statement. 

In [13]:
# 
import tabula



In [14]:
import pdfplumber

# Read pdf into DataFrame
f_path_balance= "../../data/raw/bs_favorita_2022.pdf"

def table_filter(l: list):

    _indx = np.nan

    for _i, _l in enumerate(l):
        if any(i in _l for i in ['CUENTA', 'CÓDIGO', 'VALOR (En USD$)']):
            _indx = _i
            break
    
    return l[_indx:]

df = []

with pdfplumber.open(f_path_balance) as f:
    for page in f.pages[:-1]:
        table = table_filter(page.extract_table())
        df_ = pd.DataFrame(table[1:],
                           columns=table[0]) \
                .dropna(axis=1, how='all') \
                .rename(columns={'CUENTA': 'cuenta', 
                                 'CÓDIGO': 'codigo', 
                                 'VALOR (En USD$)': 'valor'}) \
                .assign(valor=lambda df_: df_.valor.str.replace(',', '').astype(float)) 
        
        df.append(df_)
        

df = pd.concat(df, ignore_index=True)

In [15]:
df.head()

,cuenta,codigo,valor
0,ACTIVO,1,2.480404e+09
1,ACTIVO CORRIENTE,101,7.016353e+08
2,EFECTIVO Y EQUIVALENTES DE EFECTIVO,10101,7.283312e+06
3,CAJA,1010101,1.342367e+06
4,INSTITUCIONES FINANCIERAS PÚBLICAS,1010102,0.000000e+00


In [16]:
set(df['codigo'].str.len())

{1, 2, 3, 5, 7, 9, 11}

The first stage consists of getting the data. That is, to process and transform it to the point that is can stand as the cleaned up counterpart to the raw data and that is already is usable format. The processing and transformation routines must be defined.

#### Processing Tasks
- Data Extration
    - Define routines to extract data from standardized PDFs presented by institutions and companies.
    - Define routines to connect to the data generation agents in the companies
- Data Validity Checks 
    - Check for consistency in the values 
        - Substracting accounts must be negative
        - Higher level accounts must be the sum of the lower level accounts.
- Data Transformations and Aggregations

**Validity Checks**
- Check for consistency in the values 
    - Substracting accounts must be negative
    - Higher level accounts must be the sum of the lower level accounts.

In [17]:
# checking for consistency: account values' signs.

def check_negative(df: pd.DataFrame) -> pd.DataFrame:

    # assert 'valor' exists, otherwise raise error
    assert all(c in df.columns for c in ['cuenta', 'valor', 'codigo']), "Column 'valor' must be in df"

    # redefine the 'valor' column correcting for values that should be negative
    cond = (df['cuenta'].str.contains('(-)', regex=False)) & (df['valor'] > 0)    
    _valor = df.valor.mask(cond, other=df.valor.mul(-1))
    return df.assign(valor=_valor)

check_negative(df).head()

,cuenta,codigo,valor
0,ACTIVO,1,2.480404e+09
1,ACTIVO CORRIENTE,101,7.016353e+08
2,EFECTIVO Y EQUIVALENTES DE EFECTIVO,10101,7.283312e+06
3,CAJA,1010101,1.342367e+06
4,INSTITUCIONES FINANCIERAS PÚBLICAS,1010102,0.000000e+00


In [18]:
# checking for consistency: higher level accounts values must be equal to the sum of its immediate lower level accounts

# recursive addition of lower level accounts
def recursive_agg(df: pd.DataFrame, account: str) -> float:
    """Add all children accounts of a given account, considering also the children of the children\
        to ensure consistency."""
    
    assert all(c in df.columns for c in ['valor', 'codigo']), "Column 'valor' must be in df"

    account = str(account)

    # subset data for immediate children 
    cond = (df['codigo'].str.startswith(account)) & \
        (df['codigo'].str.len() == len(account) + 2)
    children = df[cond]

    # recursive addition of children accounts
    if children.empty:
        return df[df['codigo']==account].valor.sum()
    else:
        children_sum = 0
        for child in children['codigo'].values:
            children_sum += recursive_agg(df, child)

        # Adjust equity account to include account 31, a even numbered account. An anomaly.
        if account == str(3):
            children_sum += df[df['codigo']=='31'].valor.sum()  

        return round(children_sum, ndigits=2)

recursive_agg(df, 1)

2480403867.05

**Data Extraction**
- SUPERCIAS provides compressed folders with disaggregated accounts. Files in each folder vary which presents an obstacle.  
- The data that serves as the basis for the preparation of these statements is generated by many agents in the companies. Specific programmes or routines must be developed as more information is gathered. It will be important to work with clients to establish standards on data governance. 

In [19]:
from IPython.display import display
import zipfile

z_path = "../../data/raw/estadosFinancieros_2022.zip"

# read in larger, extensive dataset for all companies
with zipfile.ZipFile(z_path, mode='r') as z:
    with z.open('balances_2022_1.txt', mode='r') as zout:
        balances = pd.read_csv(zout, 
                         sep='\t', 
                         encoding='latin1',)
    with z.open('catalogo_2022_1.txt') as zout:
        balances_meta = pd.read_csv(zout, 
                         sep='\t', 
                         encoding='latin1',
                         names=['codigo', 'cuenta'])
        
display(balances.T.head(), balances_meta.head())

C:\Users\edydu\AppData\Local\Temp\ipykernel_22368\1380044315.py:9: DtypeWarning: Columns (15,17,18,19,20,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,50,51,52,53,54,55,56,57,58,59,60,61,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,80,81,82,83,84,85,86,87,88,89,90,92,93,94,95,96,97,98,99,101,102,103,104,105,106,107,108,109,110,111,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,151,152,155,164,165,166,176,180,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,204,207,209,212,213,214,215,216,217,218,219,220,221,222,224,225,226,227,228,230,231,232,236,237,239,240,241,242,245,246,253,254,258,261,262,263,264,265,266,267,274,283,284,285,287,290,291,293,296,297,298,299,300,301,302,303,304,305,306,308,310,313,316,317,318,319,322,326,327,329,330,331,332,333,334,335,336,337,338,339,340,341,342,346,347,348,349,356,357,358,359,360,362,365,366,367,368,369,370,373,374,375,376,377,378,382,387,388,391,392,394,397,398,

,0,1,2,3,4,5,6,7,8,9,...,99038,99039,99040,99041,99042,99043,99044,99045,99046,99047
AÑO,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,...,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022
EXPEDIENTE,1,2,3,11,22,40,41,49,63,76,...,749317,749354,749392,749465,749685,749940,750412,750435,750568,751125
RUC,1790013731001,1790004724001,1790008959001,1790044149001,1790023516001,1790340155001,1790046400001,1790036391001,1790084604001,1790016714001,...,1793205173001,1793205159001,1793205064001,1793204919001,891792815001,2191774183001,2191774209001,1391934838001,1793207692001,1091797737001
NOMBRE,ACEITES TROPICALES SOCIEDAD ANONIMA ATSA,ACERIA DEL ECUADOR CA ADELCA.,ACERO COMERCIAL ECUATORIANO S.A.,AEROVIAS DEL CONTINENTE AMERICANO S.A. AVIANCA,AGENCIAS Y REPRESENTACIONES CORDOVEZ SA,AGUA Y GAS DE SILLUNCHI SA,AIR FRANCE SUCURSAL EN ECUADOR,ALMACENES EL GLOBO DE QUITO SA,CONFITECA C.A.,ASOCIACION MOTELERA SA AMSA,...,COMERCIALIZADORA MABO CIA.LTDA.,SOLARBUILD S.A.,OPERATRAVEL C.LTDA.,COMPAÑIA DE TAXIS MINDOLINDOTOUR S.A.,CONSTRUCCION ECOEFICIENTE CONEFI CIA.LTDA.,COMPAÑIA RENTACAR TRANSANTACECILIA S.A.,COMPAÑIA DE TRANSPORTE PESADO PAUTA & EDA TRAN...,SYSNET-ECUADOR S.A.,ECUADOR LIFETIME EXPERIENCE LIFETIMECUADOR CIA...,RUNATRIP ECUADOR CIA.LTDA.
RAMA_ACTIVIDAD,A,C,G,H,G,C,H,G,C,I,...,G,G,N,H,M,N,H,J,N,N


,codigo,cuenta
0,1,ACTIVO
1,101,ACTIVO CORRIENTE
2,10101,EFECTIVO Y EQUIVALENTES DE EFECTIVO
3,1010101,CAJA
4,1010102,INSTITUCIONES FINANCIERAS PÚBLICAS


In [20]:
def read_member(z_path: str, file: str, **kwargs) -> pd.DataFrame:
    """Read in a file from a zip archive."""
    with zipfile.ZipFile(z_path, mode='r') as z:
        with z.open(file, mode='r') as zout:
            df = pd.read_csv(zout, 
                             sep='\t', 
                             encoding='latin1',
                             **kwargs)
    return df

**Multiple files**

**Transformation and preprocessing**


In [21]:
balances = read_member(
    z_path="../../data/raw/estadosFinancieros_2022.zip",
    file="balances_2022_1.txt"
)

balances.columns[:7]

C:\Users\edydu\AppData\Local\Temp\ipykernel_22368\3786222250.py:5: DtypeWarning: Columns (15,17,18,19,20,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,50,51,52,53,54,55,56,57,58,59,60,61,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,80,81,82,83,84,85,86,87,88,89,90,92,93,94,95,96,97,98,99,101,102,103,104,105,106,107,108,109,110,111,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,151,152,155,164,165,166,176,180,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,204,207,209,212,213,214,215,216,217,218,219,220,221,222,224,225,226,227,228,230,231,232,236,237,239,240,241,242,245,246,253,254,258,261,262,263,264,265,266,267,274,283,284,285,287,290,291,293,296,297,298,299,300,301,302,303,304,305,306,308,310,313,316,317,318,319,322,326,327,329,330,331,332,333,334,335,336,337,338,339,340,341,342,346,347,348,349,356,357,358,359,360,362,365,366,367,368,369,370,373,374,375,376,377,378,382,387,388,391,392,394,397,398,

Index(['AÑO', 'EXPEDIENTE', 'RUC', 'NOMBRE', 'RAMA_ACTIVIDAD',
       'DESCRIPCION_RAMA', 'CIIU'],
      dtype='object')

In [22]:
def check_name(c: str):
    try:
        int(c)
        return True
    except ValueError:
        return False


(balances
 .rename(lambda c: c.lower().replace('cuenta_', ''), axis=1)
 .pipe(lambda df_: df_.drop(columns=[c for c in df_.columns 
                                     if not check_name(c) and 'ruc' not in c]))
 .melt(id_vars='ruc', var_name='codigo', value_name='valor')
 .sort_values(by=['ruc', 'codigo'])
 .assign(valor=lambda df_: df_.valor.str.replace(',', '.').astype(float))
 .set_index(['ruc', 'codigo'])
)

valor
ruc           codigo             
190000222001  1        1265640.82
              101      1231938.51
              10101     209109.71
              1010101   125644.46
              1010102        0.00
...                           ...
9999999999999 80008          0.00
              80009          0.00
              801            0.00
              80101          0.00
              80102           NaN

[61607856 rows x 1 columns]

In [23]:
(balances
 .iloc[:, :7] 
 .rename(lambda c: c.lower(), axis=1)
 .drop(columns=['expediente'])
 .set_index('ruc')
)

,año,nombre,rama_actividad,descripcion_rama,ciiu
ruc,,,,,
1790013731001,2022,ACEITES TROPICALES SOCIEDAD ANONIMA ATSA,A,"AGRICULTURA, GANADERÍA, SILVICULTURA Y PESCA.",A0126.01
1790004724001,2022,ACERIA DEL ECUADOR CA ADELCA.,C,INDUSTRIAS MANUFACTURERAS.,C2410.25
1790008959001,2022,ACERO COMERCIAL ECUATORIANO S.A.,G,COMERCIO AL POR MAYOR Y AL POR MENOR REPARACIÓ...,G4610.03
1790044149001,2022,AEROVIAS DEL CONTINENTE AMERICANO S.A. AVIANCA,H,TRANSPORTE Y ALMACENAMIENTO.,H5110.01
1790023516001,2022,AGENCIAS Y REPRESENTACIONES CORDOVEZ SA,G,COMERCIO AL POR MAYOR Y AL POR MENOR REPARACIÓ...,G4630.95
...,...,...,...,...,...
2191774183001,2022,COMPAÑIA RENTACAR TRANSANTACECILIA S.A.,N,ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE ...,N7710.00
2191774209001,2022,COMPAÑIA DE TRANSPORTE PESADO PAUTA & EDA TRAN...,H,TRANSPORTE Y ALMACENAMIENTO.,H4923.01
1391934838001,2022,SYSNET-ECUADOR S.A.,J,INFORMACIÓN Y COMUNICACIÓN.,J6110.01


In [25]:
meta = read_member(
    z_path="../../data/raw/estadosFinancieros_2022.zip",
    file="catalogo_2022_1.txt",
    names=['codigo', 'cuenta'],
    index_col='codigo'

)

meta

,cuenta
codigo,
1,ACTIVO
101,ACTIVO CORRIENTE
10101,EFECTIVO Y EQUIVALENTES DE EFECTIVO
1010101,CAJA
1010102,INSTITUCIONES FINANCIERAS PÚBLICAS
...,...
80008,IMPUESTO SOBRE LAS GANANCIAS RELATIVO A OTRO R...
80009,OTROS (DETALLAR EN NOTAS)
801,RESULTADO INTEGRAL TOTAL DEL AÑO
